In [1]:
import numpy as np
import scipy 
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import matplotlib.pyplot as plt
import scipy.optimize as optim
from scipy.spatial import distance_matrix
from tqdm.notebook import tqdm

In [2]:
import pickle

with open(r"C:\Users\Alex\Desktop\misc\1-Skeleton\20200210-SolventNet_For_Bruce_v1\20200210-SolventNet_For_Bruce\Data",'rb') as f: data = pickle.load(f)

In [3]:
x_array = np.asarray(data[0])

In [4]:
def euler_char(A,thresh):
    Betti1 = []
    edge = []
    vert = []
    com = []
    As = []
    Ms = []

    for t in thresh:
        
        points = []
        for i in range(20):
            for j in range(20):
                for k in range(20):
                    points.append([i,j,k])

        dist = np.vstack(points)
        np.shape(dist)
        distmat = distance_matrix(points,points)
        M = (distmat <= 1)*1
        
        #Binary Image
        binary = (A <= t)*1
        #Adjust Adjaceny Matrix
        count = 0
        for i in range(np.shape(A)[1]):
            for j in range(np.shape(A)[0]):
                for k in range(np.shape(A)[2]):
                    if binary[j,i,k] == 0:
                        M[count,:] = 0
                        M[:,count] = 0
                    count += 1
                
        #Number Edges
        Edges = np.sum(M)/2
        
        #Number Vertices
        Vertices = np.sum(binary)
        
        removed_Vertices = np.shape(M)[0] - Vertices
        
        #Number Connected Components
        graph = csr_matrix(M)
        n_comp = connected_components(csgraph = graph, directed = False, return_labels = False)
        n_comp = n_comp - removed_Vertices
        
        #Betti 1
        B1 = n_comp - Vertices + Edges
        
        Betti1.append(B1)
        
        edge.append(Edges)
        
        vert.append(Vertices)
        
        com.append(n_comp)
        
        As.append(binary)
        
        Ms.append(M)
        
    return Betti1,com

In [5]:
def logistic(t,a,b,c):
    return c/(1+a*np.exp(-b*t))

def data_col(sigma,numgauss,points):
    allker = []
    ker = 0
    for j in range(points):
        for i in range(numgauss):
            k = np.random.uniform(-1,1,1)
            j = np.random.uniform(-1,1,1)
            norm_constant = 1 / (2 * np.pi * sigma**2)
            rhs = np.exp(-.5 * ((x + j)**2 + (y + k)**2) / sigma**2)
            kers = norm_constant * rhs
            ker = kers + ker
        ker = (ker - np.min(ker))/(np.max(ker) - np.min(ker))
        allker.append(ker)
        
    return allker

def fit_param(Data,M,thresh):
    X = np.vstack([np.ones(len(thresh)),thresh,thresh**2,thresh**3,thresh**4]).T
    psuedo = np.linalg.inv(X.T@X)@X.T
    params = []
    grads = []
    ys = []
    for i in range(len(Data)):
        M = make_matrix(U,U)
        y = np.vstack(euler_char(Data[i],U,U,thresh))
        param = psuedo@y
        params.append(param)
        grad = param[1] + param[2]*2*thresh + param[3]*thresh**2
        grads.append(np.max(grad))
        ys.append(y)
    return params,grads,ys

def fit_param_leg(Data,M,thresh):
    x = thresh
    params = []
    y = []
    for i in range(len(Data)):
        M = make_matrix(U,U)
        y = np.vstack(euler_char(Data[i],U,U,thresh))
        param = np.polynomial.legendre.legfit(thresh,y,4)
        params.append(param)
        ys.append(y)
    return params,ys

def fit_log(Data,thresh):
    x = thresh
    params = []
    ys = []
    for i in range(len(Data)):
#         M = make_matrix(U,U)
        y = np.vstack(euler_char(Data[i],thresh))
        p0 = np.random.exponential(size = 3)
        bounds = (0,[1000000000,30,20000])
        x = thresh + 1
        y = np.array(np.hstack(y))
        (a,b,c),cov = optim.curve_fit(logistic,x,y,bounds = bounds, p0=p0)
        param = [a,b,c]
        params.append(param)
        ys.append(y)
    return params,ys

In [6]:
vects = []
labels = []
i = 0
thresh = np.arange(0,1,.01)
for x in tqdm(x_array):
    x = x[:,:,:,:,0] + x[:,:,:,:,1]
    param,ys = fit_log(x,thresh)
#     vect = np.vstack(vect)
    vects.append(ys)
#     labels.append(label)
    i += 1


KeyboardInterrupt: 